In [ ]:
# REMEMBER: Heroku Postgres has a 10k rows limit
# TODO: Use another database, or use array inside row

### [Pensador](https://www.pensador.com/)

#### Libraries

In [ ]:
!pip install -U python-dotenv

In [ ]:
import os
import json
from pathlib import Path
from collections import OrderedDict

import requests
import psycopg2
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from IPython.display import clear_output

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/archive/sir-protocol-bot/scraper'

#### Scraper

In [ ]:
class PensadorScraper:
  def __init__(self):
    self.links = OrderedDict()
    self.phrases = OrderedDict()
    self.next_links = ['/']
    self.previous_links = set()
    self.home = 'https://www.pensador.com'
  
  def get_links(self, url, soup):
    links = []
    for a in soup.find_all('a'):
      try:
        if a['href'].startswith('/') and \
           a['href'].find('.php') == -1 and \
           a['href'].find('/frase') == -1 and \
           a['href'].find('/colecao') == -1:
          links.append(a['href'])
      except:
        print('a tag without href property')
    links = set(links)
    self.links[url] = list(links)
    links = links - self.previous_links
    self.next_links.extend(links)

  def get_phrases(self, url, soup):
    for card in soup.find_all('div', 'thought-card'):
      try:
        id_ = card.find('p', 'frase')['id']
        phrase_ = card.find('p', 'frase').text
        phrase_ = phrase_.strip().replace('\u2060', '')
      except:
        continue
      
      try:
        author_ = card.find('span', 'autor').a.text
        author_url_ = card.find('span', 'autor').a['href']
      except:
        author_ = ''
        author_url_ = ''
      
      try:
        n_shares_ = card.find('div', class_='total-shares')
        n_shares_ = n_shares.text.replace(' compartilhamentos', '')
        if n_shares_[-4:] == ' mil':
          n_shares_ = n_shares_.replace(' mil', '')
          n_shares_ = float(n_shares_) * 1000
        elif n_shares_:
          n_shares_ = float(n_shares_)
        else:
          n_shares_ = 0
      except:
        n_shares_ = 0
        
      try:
        img_url_ = card['data-src']
      except:
        img_url_ = ''
      
      if id_ not in self.phrases:
        self.phrases[id_] = {
          'phrase': phrase_,
          'phrase_url': f'/frase/{id_}/',
          'author': author_,
          'author_url': author_url_,
          'n_shares': n_shares_,
          'img_url': img_url_,
          'urls': [url]
        }
      elif url not in self.phrases[id_]['urls']:
        self.phrases[id_]['urls'].append(url)
  
  def work(self):
    try:
      while self.next_links:
        url = self.next_links.pop()
        self.previous_links.add(url)

        try:
          page = requests.get(self.home + url)
          page.raise_for_status()
        except requests.exceptions.HTTPError as errh:
          print('HTTP Error\n', errh)
        except requests.exceptions.ConnectionError as errc:
          print('Error Connecting:\n', errc)
        except requests.exceptions.Timeout as errt:
          print('Timeout Error:\n', errt)
        except requests.exceptions.RequestException as err:
          print('An unexpected error:\n', err)
        else:
          soup = BeautifulSoup(page.content, 'html.parser')

          self.get_phrases(url, soup)
          self.get_links(url, soup)
          
          prev_size = len(self.previous_links)
          next_size = len(self.next_links)
          print(url, f'{prev_size}:{next_size}')
          clear_output(wait=True)
    except KeyboardInterrupt:
      pass
  
  def load(self):
    with open('./phrases.json', 'r') as file:
      self.phrases = json.load(file)
    with open('./links.json', 'r') as file:
      self.links = json.load(file)
    with open('./next_links.json', 'r') as file:
      self.next_links = json.load(file)
    with open('./previous_links.json', 'r') as file:
      self.previous_links = set(json.load(file))
  
  def save(self):
    with open('./phrases.json', 'w') as file:
      json.dump(self.phrases, file)
    with open('./links.json', 'w') as file:
      json.dump(self.links, file)
    with open('./next_links.json', 'w') as file:
      json.dump(self.next_links, file)
    with open('./previous_links.json', 'w') as file:
      json.dump(list(self.previous_links), file)
  
  def remove(self):
    Path('./phrases.json').unlink()
    Path('./links.json').unlink()
    Path('./next_links.json').unlink()
    Path('./previous_links.json').unlink()

#### PostgreSQL DB

Load environment variables

In [ ]:
load_dotenv(dotenv_path='../.env')
DATABASE_URL = os.environ['DATABASE_URL'] #['DATABASE_LOCAL_URL']

Create tables

In [ ]:
try:
  conn = None
  conn = psycopg2.connect(DATABASE_URL)#, sslmode='require')
  cur = conn.cursor()
  
  # create link table
  cur.execute('''
    CREATE TABLE links (
      link_id SERIAL PRIMARY KEY,
      link VARCHAR(400) NOT NULL,
      neighbors TEXT []
    );
  ''')

  cur.close()
  conn.commit()
except (Exception, psycopg2.DatabaseError) as err:
  print(err)
finally:
  if conn is not None:
    conn.close()

In [ ]:
try:
  conn = None
  conn = psycopg2.connect(DATABASE_URL)#, sslmode='require')
  cur = conn.cursor()
  
  # create phrase table
  cur.execute('''
    CREATE TABLE phrases (
      phrase_id SERIAL PRIMARY KEY,
      phrase TEXT NOT NULL,
      phrase_url VARCHAR(200),
      author VARCHAR(200),
      author_url VARCHAR(200),
      img_url VARCHAR(400),
      n_shares INTEGER,
      urls TEXT []
    );
  ''')

  cur.close()
  conn.commit()
except (Exception, psycopg2.DatabaseError) as err:
  print(err)
finally:
  if conn is not None:
    conn.close()

Insert rows

In [ ]:
scraper = PensadorScraper()
scraper.load()

phrases = [(
  item['phrase'], 
  item['phrase_url'], 
  item['author'], 
  item['author_url'], 
  item['img_url'], 
  item['n_shares'], 
  item['urls']) 
  for _, item in scraper.phrases.items()]

links = list(scraper.links.items())

del scraper

In [ ]:
try:
  conn = None
  conn = psycopg2.connect(DATABASE_URL, sslmode='require')
  cur = conn.cursor()
  
  # insert data into phrase table
  query = ','.join(['%s'] * len(phrases))
  query = f'''
    INSERT INTO 
    phrases (phrase, phrase_url, author, author_url, img_url, n_shares, urls) 
    VALUES {query};
    '''
  query = cur.mogrify(query, phrases).decode('utf8')
  cur.execute(query)
  
  # insert data into link table
  query = ','.join(['%s'] * len(links))
  query = f'''
    INSERT INTO 
    links (link, neighbors) 
    VALUES {query};
    '''
  query = cur.mogrify(query, links).decode('utf8')
  cur.execute(query) 

  cur.close()
  conn.commit()
except (Exception, psycopg2.DatabaseError) as err:
  print(err)
finally:
  if conn is not None:
    conn.close()

Count rows

In [ ]:
try:
  try:
    conn = None
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    cur = conn.cursor()

    # select data from phrase table
    query = '''
      SELECT COUNT(link_id)
      FROM links
      '''
    cur.execute(query)

    # see phrases
    result = cur.fetchone()
    print(f'count: {result[0]}')

    cur.close()
    conn.commit()
  except (Exception, psycopg2.DatabaseError) as err:
    print(err)
  finally:
    if conn is not None:
      conn.close()
except KeyboardInterrupt:
  pass

In [ ]:
try:
  try:
    conn = None
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    cur = conn.cursor()

    # select data from phrase table
    query = '''
      SELECT COUNT(phrase_id)
      FROM phrases
      '''
    cur.execute(query)

    # see phrases
    result = cur.fetchone()
    print(f'count: {result[0]}')

    cur.close()
    conn.commit()
  except (Exception, psycopg2.DatabaseError) as err:
    print(err)
  finally:
    if conn is not None:
      conn.close()
except KeyboardInterrupt:
  pass

Consult rows

In [ ]:
try:
  try:
    conn = None
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    cur = conn.cursor()

    # select data from phrase table
    query = '''
      SELECT *
      FROM phrases'''
    cur.execute(query)

    # see phrases
    row = cur.fetchone()
    while row is not None:
      print(row)
      input()
      row = cur.fetchone()

    cur.close()
    conn.commit()
  except (Exception, psycopg2.DatabaseError) as err:
    print(err)
  finally:
    if conn is not None:
      conn.close()
except KeyboardInterrupt:
  pass

In [ ]:
try:
  try:
    conn = None
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    cur = conn.cursor()

    # select data from phrase table
    query = '''
      SELECT *
      FROM links'''
    cur.execute(query)

    # see phrases
    row = cur.fetchone()
    while row is not None:
      print(row)
      input()
      row = cur.fetchone()

    cur.close()
    conn.commit()
  except (Exception, psycopg2.DatabaseError) as err:
    print(err)
  finally:
    if conn is not None:
      conn.close()
except KeyboardInterrupt:
  pass

Drop rows

In [ ]:
try:
  try:
    conn = None
    conn = psycopg2.connect(DATABASE_URL, sslmode='require')
    cur = conn.cursor()

    # delete rows from phrase table
    query = '''
      DELETE FROM phrases
      WHERE phrase_id > 10000'''
    cur.execute(query)

    cur.close()
    conn.commit()
  except (Exception, psycopg2.DatabaseError) as err:
    print(err)
  finally:
    if conn is not None:
      conn.close()
except KeyboardInterrupt:
  pass

Drop tables

In [ ]:
try:
  conn = None
  conn = psycopg2.connect(DATABASE_URL, sslmode='require')
  cur = conn.cursor()
  
  # drop link table
  cur.execute('DROP TABLE links')

  cur.close()
  conn.commit()
except (Exception, psycopg2.DatabaseError) as err:
  print(err)
finally:
  if conn is not None:
    conn.close()

In [ ]:
try:
  conn = None
  conn = psycopg2.connect(DATABASE_URL, sslmode='require')
  cur = conn.cursor()
  
  # drop phrase table
  cur.execute('DROP TABLE phrases')

  cur.close()
  conn.commit()
except (Exception, psycopg2.DatabaseError) as err:
  print(err)
finally:
  if conn is not None:
    conn.close()

#### References

[1] www.postgresqltutorial.com

[2] https://realpython.com/beautiful-soup-web-scraper-python

[3] https://devcenter.heroku.com/articles/heroku-postgresql